In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.17.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-08-22 12:17:24--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.006s  

2024-08-22 12:17:24 (57.8 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-08-22 12:17:24--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
df_train = pd.read_table(train_file_path, header=0, names=['indicates', 'text'], usecols=['indicates', 'text'])
df_test = pd.read_table(test_file_path, header=0, names=['indicates', 'text'], usecols=['indicates', 'text'])

df_train['indicates'] = df_train['indicates'].replace("ham", 0)
df_train['indicates'] = df_train['indicates'].replace("spam", 1)
df_test['indicates'] = df_test['indicates'].replace("ham", 0)
df_test['indicates'] = df_test['indicates'].replace("spam", 1)

train_data = tf.data.Dataset.from_tensor_slices((df_train['text'].values, df_train['indicates'].values))
test_data = tf.data.Dataset.from_tensor_slices((df_test['text'].values, df_test['indicates'].values))

In [4]:
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()

for text_tensor, _ in train_data.concatenate(test_data):
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

In [5]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

In [6]:
def encode_text(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label
def map_encode_fn(text, label):
  encoded_text, label = tf.py_function(encode_text, inp=[text, label], Tout=(tf.int64, tf.int64))

  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label
train_data_encoded = train_data.map(map_encode_fn)
test_data_encoded = test_data.map(map_encode_fn)

In [7]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32
train_dataset = (train_data_encoded.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE))
test_dataset = (test_data_encoded.padded_batch(BATCH_SIZE))

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [9]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset,
    validation_steps=30)

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 32s 151ms/step - accuracy: 0.8936 - loss: 0.3738 - val_accuracy: 0.9812 - val_loss: 0.0750
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step - accuracy: 0.9868 - loss: 0.0430 - val_accuracy: 0.9745 - val_loss: 0.0637
Epoch 3/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.9953 - loss: 0.0159 - val_accuracy: 0.9885 - val_loss: 0.0491
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.9993 - loss: 0.0044 - val_accuracy: 0.9861 - val_loss: 0.0412
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.9885 - val_loss: 0.0537
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.9997 - loss: 0.0019 - val_accuracy: 0.9861 - val_loss: 0.0752
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 0.9999 - loss: 8.0357e-04 - val_accuracy: 0.9906 - val_loss: 0.0562
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.9998 - loss: 8.0373e-04 - val_accuracy: 0.9884 - val_loss: 0.0553
Epoch 9/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.9999 - loss: 3.8902e-04 - val_accuracy: 0.9917 - val_loss: 0.0538
Epoch 10/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 1.0000 - loss: 1.5001e-04 - 

In [10]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = encoder.encode(pred_text)
  pred_text = tf.cast(pred_text, tf.float32)
  prediction = model.predict(tf.expand_dims(pred_text, 0)).tolist()
  if prediction[0][0] < 0.5:
    prediction.append('ham')
  else:
    prediction.append('spam')
  return (prediction)




pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
[[-2.2379143238067627], 'ham']


In [11]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
You passed the challenge. Great job!
